In [1]:
import pandas as pd
import datetime
import numpy as np
import pymongo
import json


In [2]:
def stringToDate(string):
    string = string.split("/")
    if(len(string[2]) == 4):
        string[2] = string[2][2:]
    date = string[0]+"/"+ string[1]+"/20"+string[2]
    date = datetime.datetime.strptime(date, '%d/%m/%Y')
    return date

In [3]:
def hash_match(date, away, home, ver):
    date = date.split("/")
    return hash(date[0]+date[1]+date[2]+away+home+str(ver))    

In [4]:
def getLastMatchId(id_match, nb_earlier_match):
    dates = concats['Date'].apply(stringToDate)
    #homepart
    ids_HH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["HomeTeam"][id_match])].index
    ids_HA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["HomeTeam"][id_match])].index
    ids_home = np.array(ids_HH,ids_HA)
    if(ids_home.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_home = ids_home[ids_home.argsort()[-nb_earlier_match:][::-1]]
    #awaypart
    
    ids_AH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["AwayTeam"][id_match])].index
    ids_AA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["AwayTeam"][id_match])].index
    ids_away = np.array(ids_AH,ids_AA)
    if(ids_away.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_away = ids_away[ids_away.argsort()[-nb_earlier_match:][::-1]]
    return np.array([ids_away, ids_home])

In [5]:
concats = pd.DataFrame()
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("data/"+str(i)+"1.csv")], sort = False,ignore_index=True)
    
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("data/"+str(i)+"2.csv")], sort = False,ignore_index=True)

In [6]:
concats = concats[pd.notnull(concats['FTR'])]
concats = concats.drop(columns = concats.columns[22:])
concats['ID_FTR'] = concats.FTR.astype('category').cat.codes

In [7]:
hashmap = dict()

In [ ]:
with open('hm.pickle', 'rb') as handle:
    hashmap = pickle.load(handle)

In [ ]:
ids = []
for i in range(concats.shape[0]-500):
    vals = concats.iloc[i,[1,2,3]].values
    hashed = hash_match(vals[0],vals[1], vals[2], 1)
    if hashed in hashmap:
        ids.append(hashmap[hashed])
    else:
        id_match = getLastMatchId(concats.index[i],3)
        ids.append(id_match)
        hashmap[hashed] = id_match


In [ ]:
hashmap

In [ ]:
with open('hm.pickle', 'wb') as handle:
    pickle.dump(hashmap, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [200]:
ids = pd.DataFrame(ids)
ids = ids.reindex(concats.index)
ids = ids.dropna()

In [201]:
concats = concats.fillna(0.0)

In [202]:


concats['AAG'] = np.nan
concats['AAS'] = np.nan
concats['AASO'] = np.nan
concats['AAF'] = np.nan
concats['AAC'] = np.nan
concats['AAY'] = np.nan
concats['AAR'] = np.nan

concats['HAG'] = np.nan
concats['HAS'] = np.nan
concats['HASO'] = np.nan
concats['HAF'] = np.nan
concats['HAC'] = np.nan
concats['HAY'] = np.nan
concats['HAR'] = np.nan

for i in ids.index:
    #awaypart
    ids_away = ids[0][i][0]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_away:
        if(concats.loc[i,"AwayTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
            
    concats.loc[i,'AAG'] = avg_goals
    concats.loc[i,'AAS'] = avg_shots
    concats.loc[i,'AASO'] = avg_shots_on
    concats.loc[i,'AAF'] = avg_fouls
    concats.loc[i,'AAC'] = avg_corners
    concats.loc[i,'AAY'] = avg_yellow
    concats.loc[i,'AAR'] = avg_red

            
    ids_home = ids[0][i][1]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_home:
        if(concats.loc[i,"HomeTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
        
       
    
    concats.loc[i,'HAG'] = avg_goals
    concats.loc[i,'HAS'] = avg_shots
    concats.loc[i,'HASO'] = avg_shots_on
    concats.loc[i,'HAF'] = avg_fouls
    concats.loc[i,'HAC'] = avg_corners
    concats.loc[i,'HAY'] = avg_yellow
    concats.loc[i,'HAR'] = avg_red

In [203]:
concats = concats.drop([x for x in concats.index.values if x not in ids.index.values])

In [204]:
concats = concats.drop(columns = concats.columns[4:6])
concats = concats.drop(columns = concats.columns[5:20])

In [205]:
normalized_df=(concats[concats.columns[6:]]-concats[concats.columns[6:]].mean())/concats[concats.columns[6:]].std()

In [206]:
concats[concats.columns[6:]] = normalized_df

In [207]:
concats.columns[6:13]

Index(['AAG', 'AAS', 'AASO', 'AAF', 'AAC', 'AAY', 'AAR'], dtype='object')

In [208]:
concats.columns[13:]

Index(['HAG', 'HAS', 'HASO', 'HAF', 'HAC', 'HAY', 'HAR'], dtype='object')

In [209]:
ids_team = list()
names = concats["HomeTeam"].drop_duplicates()
for i in names:
    df = np.array(concats[concats["HomeTeam"] == i].index, concats[concats["AwayTeam"] == i].index)
    ids_team.append(max(df))

values = list()
for i in range(len(names)):
    if(concats.loc[ids_team[i], "HomeTeam"] == names.iloc[i]):
        values.append(concats.loc[ids_team[i], concats.columns[13:]].values.astype(float))
    else:
        values.append(concats.loc[ids_team[i], concats.columns[6:13]].values.astype(float))
        
team_data = pd.DataFrame(values)
team_data["TeamName"] =  names.values

In [215]:
concats["ID_FTR"].value_counts()

2    1747
0    1123
1    1087
Name: ID_FTR, dtype: int64

In [211]:
team_data.to_csv("teamfeatures.csv")

In [212]:
concats.to_csv("parsed_train_data.csv")

In [15]:
def pandas_to_mongo(df):
    client = pymongo.MongoClient("mongodb://plp_mongodb:27017")
    db = client['refined-database']
    df.to_json('yourjson.json',orient = "index")                               # saving to json file
    jdf = open('yourjson.json').read()                        # loading the json file 
    data = json.loads(jdf)   
    posts = db.posts
    entry_list = list()
    for i in data.keys():
        entry_list.append(data[i])
    return posts.insert_many(entry_list)
    

In [193]:
pandas_to_mongo(team_data)
pandas_to_mongo(concats)

ServerSelectionTimeoutError: plp_mongodb:27017: [Errno 11001] getaddrinfo failed